In [0]:
dbutils.fs.ls("/mnt/bronze/dbo")

[FileInfo(path='dbfs:/mnt/bronze/dbo/Badges/', name='Badges/', size=0, modificationTime=1702955508000),
 FileInfo(path='dbfs:/mnt/bronze/dbo/Comments/', name='Comments/', size=0, modificationTime=1702956412000),
 FileInfo(path='dbfs:/mnt/bronze/dbo/LinkTypes/', name='LinkTypes/', size=0, modificationTime=1702955423000),
 FileInfo(path='dbfs:/mnt/bronze/dbo/PostLinks/', name='PostLinks/', size=0, modificationTime=1702956203000),
 FileInfo(path='dbfs:/mnt/bronze/dbo/PostTypes/', name='PostTypes/', size=0, modificationTime=1702955342000),
 FileInfo(path='dbfs:/mnt/bronze/dbo/Posts/', name='Posts/', size=0, modificationTime=1702957475000),
 FileInfo(path='dbfs:/mnt/bronze/dbo/Users/', name='Users/', size=0, modificationTime=1702955801000),
 FileInfo(path='dbfs:/mnt/bronze/dbo/VoteTypes/', name='VoteTypes/', size=0, modificationTime=1702955342000),
 FileInfo(path='dbfs:/mnt/bronze/dbo/Votes/', name='Votes/', size=0, modificationTime=1702956152000)]

Get all filenames in an array

In [0]:
table_names = []

root_path = "/mnt/bronze/dbo"

for i in dbutils.fs.ls(root_path):
    table_names.append(i.name.split('/')[0])
table_names

['Badges',
 'Comments',
 'LinkTypes',
 'PostLinks',
 'PostTypes',
 'Posts',
 'Users',
 'VoteTypes',
 'Votes']

In [0]:
table_names.remove('Users')

##Remove all HTML tags

In [0]:
import re
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

cols_with_html = ['Body', 'AboutMe']

In [0]:
# Function to remove HTML tags using regular expressions
def remove_html_tags(text):
    clean = re.sub(r'<[^>]*>', '', text)
    return clean

In [0]:

for i in table_names:
    file = f'/mnt/bronze/dbo/{i}/{i}.parquet'
    df = spark.read.format('parquet').load(file)
    column = df.columns
    print(f"\nTable name: {i}")
    # Define the remove_html_tags function as a UDF
    remove_html_tags_udf = udf(remove_html_tags, StringType())  # Assuming remove_html_tags returns a string

    for col in df.columns:
        if col in cols_with_html:
            df = df.withColumn(col, remove_html_tags_udf(col))  # Use withColumn to apply the UDF
    df.show(5)
    
    # Write to silver container in the data lake
    output_path = f'/mnt/silver/dbo/{i}'
    df.write.format('delta').mode("append").save(output_path)


Table name: Badges
+-----+-------+------+--------------------+
|   Id|   Name|UserId|                Date|
+-----+-------+------+--------------------+
|82946|Teacher|  3718|2008-09-15 08:55:...|
|82947|Teacher|   994|2008-09-15 08:55:...|
|82949|Teacher|  3893|2008-09-15 08:55:...|
|82950|Teacher|  4591|2008-09-15 08:55:...|
|82951|Teacher|  5196|2008-09-15 08:55:...|
+-----+-------+------+--------------------+
only showing top 5 rows


Table name: Comments
+---+--------------------+------+-----+--------------------+------+
| Id|        CreationDate|PostId|Score|                Text|UserId|
+---+--------------------+------+-----+--------------------+------+
|  2|2008-09-06 08:09:...| 35314|    8|Yeah, I didn't be...|     3|
|  4|2008-09-06 08:42:...| 35195|    0|I don't see an ac...|   380|
|  9|2008-09-06 12:26:...| 47239|    0|Jonathan: Wow!  T...|  4550|
| 10|2008-09-06 13:38:...| 45651|    6|It will help if y...|   242|
| 12|2008-09-06 13:51:...| 47428|    3|One of the things...| 